In [2]:
import math
from math import pi, pow, sin, cos, asin, sqrt, floor
import pandas as pd
from scipy import stats
import itertools
import numpy as np
from pyproj import Proj
import pulp
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary

In [3]:
def degrees_to_radians(x):
     return((pi/180)*x)
     
def lon_lat_distance_miles(lon_a,lat_a,lon_b,lat_b):
    radius_of_earth = 24872/(2*pi)
    c = sin((degrees_to_radians(lat_a) - \
    degrees_to_radians(lat_b))/2)**2 + \
    cos(degrees_to_radians(lat_a)) * \
    cos(degrees_to_radians(lat_b)) * \
    sin((degrees_to_radians(lon_a) - \
    degrees_to_radians(lon_b))/2)**2
    return(2 * radius_of_earth * (asin(sqrt(c))))    

def lon_lat_distance_meters (lon_a,lat_a,lon_b,lat_b):
    return(lon_lat_distance_miles(lon_a,lat_a,lon_b,lat_b) * 1609.34) 
    
# UTM zone may be computed directly from the list of longitude values
def findZone(listOfLon):
    zones = [ ((floor((long + 180)/6) ) % 60) + 1 for long in listOfLon]
    zone = stats.mode(zones, keepdims = False)[0].astype(int)
    return(zone.item())  

In [4]:
df_michigan = pd.read_excel('Michigan_Stats.xlsx')

In [5]:
df_michigan

,Full County,County,2024 Population,Growth since 2020,Area,Density,lat,lng,Republican,Democrat
0,Wayne County,Wayne,1743396,-0.0258,236,2849,42.2819,-83.2822,264553,597170
1,Oakland County,Oakland,1269827,-0.0021,335,1465,42.6604,-83.3858,325971,434148
2,Macomb County,Macomb,875967,-0.0049,185,1829,42.6957,-82.9323,263863,223952
3,Kent County,Kent,663788,0.0080,328,782,43.0322,-85.5493,165741,187915
4,Genesee County,Genesee,401121,-0.0115,246,630,43.0217,-83.7067,98714,119390
...,...,...,...,...,...,...,...,...,...,...
78,Baraga County,Baraga,8337,0.0239,347,9,46.6626,-88.3652,2512,1478
79,Schoolcraft County,Schoolcraft,8124,0.0083,452,7,46.1966,-86.1996,3090,1589
80,Luce County,Luce,6596,0.0287,347,7,46.4707,-85.5444,2109,842
81,Ontonagon County,Ontonagon,5891,0.0150,506,4,46.6644,-89.3148,2358,1391


In [6]:
counties = [
    "Wayne", "Oakland", "Macomb", "Kent", "Genesee", "Washtenaw", "Ottawa", "Ingham",
    "Kalamazoo", "Livingston", "Saginaw", "Muskegon", "St. Clair", "Jackson", "Monroe",
    "Berrien", "Calhoun", "Allegan", "Eaton", "Bay", "Lenawee", "Grand Traverse", "Lapeer",
    "Midland", "Clinton", "Van Buren", "Montcalm", "Shiawassee", "Marquette", "Ionia",
    "Barry", "Isabella", "St. Joseph", "Tuscola", "Cass", "Newaygo", "Branch", "Hillsdale",
    "Gratiot", "Mecosta", "Sanilac", "Houghton", "Delta", "Chippewa", "Wexford", "Emmet",
    "Clare", "Huron", "Mason", "Alpena", "Oceana", "Charlevoix", "Cheboygan", "Gladwin",
    "Dickinson", "Otsego", "Manistee", "Iosco", "Antrim", "Roscommon", "Osceola", "Leelanau",
    "Menominee", "Ogemaw", "Kalkaska", "Benzie", "Missaukee", "Arenac", "Gogebic", "Crawford",
    "Presque Isle", "Lake", "Iron", "Mackinac", "Alcona", "Montmorency", "Alger", "Oscoda",
    "Baraga", "Schoolcraft", "Luce", "Ontonagon", "Keweenaw"
]

populations = {
    "Wayne": 1743396, "Oakland": 1269827, "Macomb": 875967, "Kent": 663788, "Genesee": 401121,
    "Washtenaw": 364568, "Ottawa": 305826, "Ingham": 285797, "Kalamazoo": 263538, "Livingston": 197380,
    "Saginaw": 187191, "Muskegon": 177589, "St. Clair": 160080, "Jackson": 159090, "Monroe": 154855,
    "Berrien": 151687, "Calhoun": 133308, "Allegan": 122588, "Eaton": 108786, "Bay": 102362,
    "Lenawee": 96760, "Grand Traverse": 96544, "Lapeer": 89322, "Midland": 84316, "Clinton": 79788,
    "Van Buren": 75927, "Montcalm": 69099, "Shiawassee": 68083, "Marquette": 67450, "Ionia": 65688,
    "Barry": 64214, "Isabella": 63870, "St. Joseph": 60978, "Tuscola": 52744, "Cass": 51807,
    "Newaygo": 51488, "Branch": 45928, "Hillsdale": 45439, "Gratiot": 41632, "Mecosta": 41416,
    "Sanilac": 40254, "Houghton": 38038, "Delta": 36799, "Chippewa": 36321, "Wexford": 34226,
    "Emmet": 34031, "Clare": 31325, "Huron": 30709, "Mason": 29000, "Alpena": 28975,
    "Oceana": 27042, "Charlevoix": 26145, "Cheboygan": 26132, "Gladwin": 26085, "Dickinson": 26021,
    "Otsego": 25933, "Manistee": 25759, "Iosco": 25273, "Antrim": 24553, "Roscommon": 24044,
    "Osceola": 23381, "Leelanau": 23155, "Menominee": 22665, "Ogemaw": 21061, "Kalkaska": 18736,
    "Benzie": 18555, "Missaukee": 15390, "Arenac": 15167, "Gogebic": 14071, "Crawford": 13642,
    "Presque Isle": 13227, "Lake": 12817, "Iron": 11775, "Mackinac": 10740, "Alcona": 10581,
    "Montmorency": 9799, "Alger": 8741, "Oscoda": 8699, "Baraga": 8337, "Schoolcraft": 8124,
    "Luce": 6596, "Ontonagon": 5891, "Keweenaw": 2189
}


Republican_Votes = {
    "Wayne": 264553, "Oakland": 325971, "Macomb": 263863, "Kent": 165741, "Genesee": 98714,
    "Washtenaw": 56241, "Ottawa": 100913, "Ingham": 47639, "Kalamazoo": 56823, "Livingston": 76982,
    "Saginaw": 50785, "Muskegon": 45133, "St. Clair": 59185, "Jackson": 47372, "Monroe": 52710,
    "Berrien": 43519, "Calhoun": 36221, "Allegan": 41392, "Eaton": 31798, "Bay": 33125,
    "Lenawee": 31541, "Grand Traverse": 30502, "Lapeer": 35482, "Midland": 27675, "Clinton": 25098,
    "Van Buren": 21591, "Montcalm": 21815, "Shiawassee": 23149, "Marquette": 16286, "Ionia": 20657,
    "Barry": 23471, "Isabella": 14815, "St. Joseph": 18127, "Tuscola": 20297, "Cass": 16699,
    "Newaygo": 18857, "Branch": 14064, "Hillsdale": 17037, "Gratiot": 12102, "Mecosta": 13267,
    "Sanilac": 16194, "Houghton": 10378, "Delta": 13207, "Chippewa": 10681, "Wexford": 12102,
    "Emmet": 12135, "Clare": 10861, "Huron": 12731, "Mason": 10207, "Alpena": 10686,
    "Oceana": 8892, "Charlevoix": 9841, "Cheboygan": 10186, "Gladwin": 9893, "Dickinson": 9617,
    "Otsego": 9779, "Manistee": 8321, "Iosco": 9759, "Antrim": 9748, "Roscommon": 9670,
    "Osceola": 8928, "Leelanau": 7916, "Menominee": 8117, "Ogemaw": 8253, "Kalkaska": 7436,
    "Benzie": 6601, "Missaukee": 6648, "Arenac": 5928, "Gogebic": 4600, "Crawford": 5087,
    "Presque Isle": 5342, "Lake": 3946, "Iron": 4216, "Mackinac": 4304, "Alcona": 4848,
    "Montmorency": 4171, "Alger": 3014, "Oscoda": 3466, "Baraga": 2512, "Schoolcraft": 3090,
    "Luce": 2109, "Ontonagon": 2358, "Keweenaw": 862
}

Democratic_Votes = {
    "Wayne": 597170, "Oakland": 434148, "Macomb": 223952, "Kent": 187915, "Genesee": 119390,
    "Washtenaw": 157136, "Ottawa": 64705, "Ingham": 94212, "Kalamazoo": 83686, "Livingston": 48220,
    "Saginaw": 51088, "Muskegon": 45643, "St. Clair": 31363, "Jackson": 31995, "Monroe": 32975,
    "Berrien": 37438, "Calhoun": 28877, "Allegan": 24449, "Eaton": 31299, "Bay": 26151,
    "Lenawee": 20918, "Grand Traverse": 28683, "Lapeer": 16367, "Midland": 20493, "Clinton": 21968,
    "Van Buren": 16803, "Montcalm": 9703, "Shiawassee": 15347, "Marquette": 20465, "Ionia": 10901,
    "Barry": 11797, "Isabella": 14072, "St. Joseph": 9262, "Tuscola": 8712, "Cass": 9130,
    "Newaygo": 7873, "Branch": 6159, "Hillsdale": 5883, "Gratiot": 6693, "Mecosta": 7375,
    "Sanilac": 5966, "Houghton": 7750, "Delta": 7606, "Chippewa": 6648, "Wexford": 5838,
    "Emmet": 9662, "Clare": 5199, "Huron": 5490, "Mason": 6802, "Alpena": 6000,
    "Oceana": 4944, "Charlevoix": 6939, "Cheboygan": 5437, "Gladwin": 4524, "Dickinson": 4744,
    "Otsego": 4743, "Manistee": 6107, "Iosco": 5373, "Antrim": 5960, "Roscommon": 5166,
    "Osceola": 3214, "Leelanau": 8795, "Menominee": 4316, "Ogemaw": 3475, "Kalkaska": 3002,
    "Benzie": 5480, "Missaukee": 1967, "Arenac": 2774, "Gogebic": 3570, "Crawford": 2672,
    "Presque Isle": 2911, "Lake": 2288, "Iron": 2493, "Mackinac": 2632, "Alcona": 2142,
    "Montmorency": 1628, "Alger": 2053, "Oscoda": 1342, "Baraga": 1478, "Schoolcraft": 1589,
    "Luce": 842, "Ontonagon": 1391, "Keweenaw": 672
}


In [7]:
county_pairs = list(itertools.product(counties, repeat=2))
county_distances = {}

for (county1, county2) in county_pairs:
    lat1 =  df_michigan.loc[df_michigan['County'] == county1, 'lat'].values[0]
    lon1 = df_michigan.loc[df_michigan['County'] == county1, 'lng'].values[0]
    lat2 = df_michigan.loc[df_michigan['County'] == county2, 'lat'].values[0]
    lon2 = df_michigan.loc[df_michigan['County'] == county2, 'lng'].values[0]
    distance = lon_lat_distance_miles(lat1, lon1, lat2, lon2)
    county_distances[(county1, county2)] = round(distance,2)

# Now, county_distances dictionary holds the distances between each pair
print(county_distances)

{('Wayne', 'Wayne'): 0.0, ('Wayne', 'Oakland'): 7.77, ('Wayne', 'Macomb'): 24.42, ('Wayne', 'Kent'): 156.71, ('Wayne', 'Genesee'): 29.89, ('Wayne', 'Washtenaw'): 38.46, ('Wayne', 'Ottawa'): 187.55, ('Wayne', 'Ingham'): 75.43, ('Wayne', 'Kalamazoo'): 155.38, ('Wayne', 'Livingston'): 43.55, ('Wayne', 'Saginaw'): 53.87, ('Wayne', 'Muskegon'): 198.37, ('Wayne', 'St. Clair'): 41.93, ('Wayne', 'Jackson'): 78.84, ('Wayne', 'Monroe'): 17.99, ('Wayne', 'Berrien'): 216.26, ('Wayne', 'Calhoun'): 119.07, ('Wayne', 'Allegan'): 180.07, ('Wayne', 'Eaton'): 107.53, ('Wayne', 'Bay'): 50.25, ('Wayne', 'Lenawee'): 54.26, ('Wayne', 'Grand Traverse'): 158.19, ('Wayne', 'Lapeer'): 7.78, ('Wayne', 'Midland'): 77.07, ('Wayne', 'Clinton'): 91.28, ('Wayne', 'Van Buren'): 189.08, ('Wayne', 'Montcalm'): 129.41, ('Wayne', 'Shiawassee'): 59.94, ('Wayne', 'Marquette'): 301.84, ('Wayne', 'Ionia'): 123.92, ('Wayne', 'Barry'): 140.05, ('Wayne', 'Isabella'): 108.52, ('Wayne', 'St. Joseph'): 155.16, ('Wayne', 'Tuscola'):

In [24]:
import pulp
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary

districts = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']

distances = county_distances
total_population = sum(populations.values())
num_districts = len(districts)
target_population = total_population / num_districts

# Create the problem
prob = LpProblem("Redistricting", LpMinimize)

# Decision variables
x = LpVariable.dicts("x", ((county, district) for county in counties for district in districts), cat='Binary')
y = LpVariable.dicts("y", ((i, k) for i in counties for k in counties if i < k), cat='Binary')

# Objective function: Minimize the weighted sum of distances between counties in the same district
prob += lpSum(distances[(i, k)] * y[i, k] for i in counties for k in counties if i < k)

# Constraints
# Each county must be assigned to exactly one district
for county in counties:
    prob += lpSum(x[county, district] for district in districts) == 1, f"OneDistrict_{county}"

# Population balance
tolerance_pop = 1000000
for district in districts:
    prob += lpSum(populations[county] * x[county, district] for county in counties) <= target_population + tolerance_pop, f"PopUpper_{district}"
    prob += lpSum(populations[county] * x[county, district] for county in counties) >= target_population - tolerance_pop, f"PopLower_{district}"

# Contiguity constraint: this ensures y[i, k] is set properly based on x
for i in counties:
    for k in counties:
        if i < k:
            for district in districts:
                prob += y[i, k] <= x[i, district] + x[k, district] - 1, f"Contiguity_{i}_{k}_{district}"


# Solve the problem
prob.solve()

# Output results
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)


x_('Alcona',_'1') = 0.5
x_('Alcona',_'10') = 5045.7209
x_('Alcona',_'12') = 1.0
x_('Alcona',_'13') = 1.0
x_('Alger',_'1') = 0.5
x_('Alger',_'10') = 5045.7209
x_('Alger',_'12') = 1.0
x_('Alger',_'13') = 1.0
x_('Allegan',_'1') = 0.5
x_('Allegan',_'12') = 1.0
x_('Allegan',_'4') = 8.4474949
x_('Alpena',_'1') = 0.5
x_('Alpena',_'12') = 1.0
x_('Alpena',_'5') = 1.0
x_('Antrim',_'1') = 0.5
x_('Antrim',_'10') = 5045.7209
x_('Antrim',_'12') = 1.0
x_('Antrim',_'13') = 1.0
x_('Antrim',_'2') = 1.0
x_('Arenac',_'1') = 0.5
x_('Arenac',_'10') = 10247.484
x_('Arenac',_'12') = 1.0
x_('Arenac',_'13') = 1.0
x_('Arenac',_'8') = 2318.6006
x_('Baraga',_'1') = 0.5
x_('Baraga',_'10') = 5045.7209
x_('Baraga',_'12') = 1.0
x_('Barry',_'1') = 0.5
x_('Barry',_'10') = 12568.076
x_('Barry',_'12') = 1.0
x_('Barry',_'8') = 1.0
x_('Bay',_'1') = 0.5
x_('Bay',_'10') = 5045.7209
x_('Bay',_'12') = 1.0
x_('Bay',_'13') = 1.0
x_('Bay',_'2') = 1.0
x_('Benzie',_'1') = 0.5
x_('Benzie',_'10') = 10247.484
x_('Benzie',_'12') = 1.0
x

In [23]:
import pulp
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary

districts = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']

distances = county_distances
total_population = sum(populations.values())
num_districts = len(districts)
target_population = total_population / num_districts

# Create the problem
prob = LpProblem("Redistricting", LpMinimize)

# Decision variables
x = LpVariable.dicts("x", ((county, district) for county in counties for district in districts), cat='Binary')
y = LpVariable.dicts("y", ((i, k) for i in counties for k in counties if i < k), cat='Binary')

# Objective function: Minimize the weighted sum of distances between counties in the same district
prob += lpSum(distances[(i, k)] * y[i, k] for i in counties for k in counties if i < k)

# Constraints
# Each county must be assigned to exactly one district
for county in counties:
    prob += lpSum(x[county, district] for district in districts) == 1, f"OneDistrict_{county}"

# Population balance
tolerance_pop = 1000000
for district in districts:
    prob += lpSum(populations[county] * x[county, district] for county in counties) <= target_population + tolerance_pop, f"PopUpper_{district}"
    prob += lpSum(populations[county] * x[county, district] for county in counties) >= target_population - tolerance_pop, f"PopLower_{district}"

# Add political balance constraints
tolerance_pol = 400000  # Adjust tolerance as needed based on desired strictness
for district in districts:
    prob += lpSum(Republican_Votes[county] * x[county, district] - Democratic_Votes[county] * x[county, district] for county in counties) <= tolerance_pol, f"PoliticalBalanceUpper_{district}"
    prob += lpSum(Democratic_Votes[county] * x[county, district] - Republican_Votes[county] * x[county, district] for county in counties) <= tolerance_pol, f"PoliticalBalanceLower_{district}"


# Contiguity constraint: this ensures y[i, k] is set properly based on x
#for i in counties:
#    for k in counties:
#        if i < k:
#            for district in districts:
#                prob += y[i, k] <= x[i, district] + x[k, district] - 1, f"Contiguity_{i}_{k}_{district}"



# Solve the problem
prob.solve()

# Output results
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)


x_('Alcona',_'1') = 1.0
x_('Alger',_'1') = 1.0
x_('Allegan',_'10') = 1.0
x_('Alpena',_'1') = 1.0
x_('Antrim',_'1') = 1.0
x_('Arenac',_'1') = 1.0
x_('Baraga',_'1') = 1.0
x_('Barry',_'10') = 1.0
x_('Bay',_'1') = 1.0
x_('Benzie',_'3') = 1.0
x_('Berrien',_'1') = 1.0
x_('Branch',_'10') = 1.0
x_('Calhoun',_'1') = 1.0
x_('Cass',_'1') = 1.0
x_('Charlevoix',_'1') = 1.0
x_('Cheboygan',_'1') = 1.0
x_('Chippewa',_'1') = 1.0
x_('Clare',_'1') = 1.0
x_('Clinton',_'10') = 1.0
x_('Crawford',_'1') = 1.0
x_('Delta',_'1') = 1.0
x_('Dickinson',_'1') = 1.0
x_('Eaton',_'13') = 1.0
x_('Emmet',_'1') = 1.0
x_('Genesee',_'11') = 1.0
x_('Gladwin',_'1') = 1.0
x_('Gogebic',_'10') = 1.0
x_('Grand_Traverse',_'10') = 1.0
x_('Gratiot',_'1') = 1.0
x_('Hillsdale',_'10') = 1.0
x_('Houghton',_'1') = 1.0
x_('Huron',_'10') = 1.0
x_('Ingham',_'12') = 1.0
x_('Ionia',_'1') = 1.0
x_('Iosco',_'1') = 1.0
x_('Iron',_'11') = 1.0
x_('Isabella',_'11') = 1.0
x_('Jackson',_'10') = 1.0
x_('Kalamazoo',_'10') = 1.0
x_('Kalkaska',_'1') = 1.